# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio project',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [12]:
select_relevant_links("https://metrofarm.cz")

Selecting relevant links for https://metrofarm.cz by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://metrofarm.cz/'},
  {'type': 'about page', 'url': 'https://metrofarm.cz/o-nas/'},
  {'type': 'company page', 'url': 'https://metrofarm.cz/metrofarm/'},
  {'type': 'projects page', 'url': 'https://metrofarm.cz/metropole/'},
  {'type': 'projects page', 'url': 'https://metrofarm.cz/nove-projekty/'},
  {'type': 'location page', 'url': 'https://metrofarm.cz/letnany/'},
  {'type': 'project page', 'url': 'https://metrofarm.cz/guerilla-gardening/'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/MetroFarm-1308581092618428'},
  {'type': 'instagram profile',
   'url': 'https://www.instagram.com/metrofarma/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
7 days ago
•
286k
•
2.75k
zai-org/GLM-4.6V-Flash
Updated
6 days ago
•
84.2k
•
438
microsoft/VibeVoice-Realtime-0.5B
Updated
3 days ago
•
143k
•
854
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
1 day ago
•
21.4k
•
352
zai-org/GLM-4.6V
Updated
7 days ago
•
3.68k
•
309
Browse 1M+ models
Spaces
Running
on
Zero
584
Z Image Turbo
🖼
584
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.38k
Z Image Turbo
🏃
1.38k
Generate images from text prompts
Running
on
Zero
MCP
Featured
149

In [20]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n7 days ago\n•\n286k\n•\n2.75k\nzai-org/GLM-4.6V-Flash\nUpdated\n6 days ago\n•\n84.2k\n•\n438\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n3 days ago\n•\n143k\n•\n854\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\n1 day ago\n•\n21.4k\n•\n352\nzai-org/GLM-4.6V\nUpdated\n7 days ago\n•\n3.68k\n•\n309\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n584\

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face – Building the Future of AI Together

---

## About Hugging Face

Hugging Face is the vibrant AI and machine learning community platform where innovation thrives through collaboration. It serves as the central hub for hosting, sharing, and discovering over 1 million machine learning models, 250,000 datasets, and 400,000 applications across various modalities including text, image, video, audio, and even 3D. Hugging Face empowers developers, researchers, and organizations to accelerate the development and deployment of state-of-the-art AI solutions.

---

## Our Platform

- **Model Hub**: Access and contribute to over 1M open-source machine learning models spanning NLP, computer vision, audio processing, and more.
- **Datasets**: Explore and share a vast repository of datasets essential for training and benchmarking models.
- **Spaces**: Deploy, demo, and experiment with AI applications easily using Hugging Face Spaces — supporting seamless collaboration.
- **Compute & Enterprise Solutions**: For teams and enterprises, Hugging Face offers enhanced compute resources and dedicated support for scaling AI efforts efficiently.

---

## Community & Collaboration

At the core of Hugging Face is a thriving community committed to open science and ethical AI. The platform fosters:

- **Open collaboration** on research, models, and applications.
- Sharing of knowledge through detailed blogs and tutorials covering machine learning advancements, ethics, novel approaches, and case studies.
- Community-driven initiatives encouraging innovation in subfields such as NLP, reinforcement learning, robotics, and more.
- Democratizing AI technology and knowledge to enable a more responsible and inclusive future.

---

## Company Culture

Hugging Face is more than just a platform—it's a mission-driven organization focused on:

- **Openness**: Fully embracing open-source principles and transparency in AI development.
- **Ethics**: Actively promoting responsible AI that respects fairness, inclusivity, and safety.
- **Innovation**: Encouraging experimentation and continuous learning through community-led research and shared resources.
- **Collaboration**: Building a supportive ecosystem where individuals, startups, academia, and enterprises unite to push the boundaries of machine learning.

---

## Customers & Use Cases

Hugging Face supports a broad range of users including:

- Independent machine learning practitioners building portfolios and sharing models.
- Researchers leveraging open datasets and models for scientific discoveries.
- Enterprises integrating AI into products and services with scalable compute and enterprise-grade support.
- AI startups accelerating innovation while minimizing infrastructure overhead.
- Educational institutions using the platform as a learning resource and collaboration hub.

---

## Careers at Hugging Face

Join a fast-growing, mission-driven company that values creativity, initiative, and impact. With roles spanning engineering, research, product management, and community engagement, Hugging Face welcomes talent passionate about shaping the future of AI with openness and ethics.

Explore current openings and become part of a team working at the forefront of machine learning:  
**[Hugging Face Careers](https://huggingface.co/careers)**

---

## Brand & Identity

The Hugging Face brand reflects its mission of openness and community. With a friendly yellow (#FFD21E) and energetic orange (#FF9D00) color palette, the brand conveys accessibility and innovation. Logos and assets are openly available to support consistency and broad usage in the community.

---

## Learn & Connect

Stay updated with the latest AI research, tutorials, and community highlights through the Hugging Face Blog, featuring content on:

- Advanced ML techniques
- Real-world case studies
- Open-source collaboration stories
- Ethical AI guidelines and innovations

Explore and contribute: **[Hugging Face Blog](https://huggingface.co/blog)**

---

## Get Started Today

Become part of the AI revolution. Whether you want to build, share, or leverage machine learning, Hugging Face offers the tools, community, and resources to accelerate your journey toward a more intelligent future.

Visit: **[huggingface.co](https://huggingface.co)**

---

*Hugging Face – The AI community building the future, together.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face: Where AI and Fun Hug It Out 🤗

Welcome to **Hugging Face**, the *ultimate AI community* that’s not just building the future — it’s hugging it tight! If you’ve ever dreamed of a place where machine learning wizards, data magicians, and app alchemists gather to create magic, this is it.

---

## Who Are We?

Think of us as the bustling AI playground and workshop combined. We offer a vibrant **collaboration platform** where you can:

- **Discover 1M+ AI models** — from image creators to voice transformers.
- **Dive into 250k+ datasets** — hugely varied, fresh, and always intriguing.
- **Experiment on 400k+ applications** — maybe you’ll build the next viral AI app?
- Share your work and even build your AI portfolio — brag responsibly!

All this is powered by a **fast-growing, friendly community** where experts, newcomers, and enthusiasts mingle, tinker, and contribute to an ethical, open-source AI future.

---

## Behind the Screens: What You Can Explore

### Models Trending This Week
- **Tongyi-MAI/Z-Image-Turbo**: Image generation on turbo mode 🚀 (286k downloads, 2.75k loves)
- **microsoft/VibeVoice-Realtime-0.5B**: Real-time voice manipulation, because who needs real voices anyway? 🎤
- **mistralai/Devstral-Small-2-24B-Instruct-2512**: Small but mighty NLP superstar 🔮

### Spaces: The Creative Labs
- Generate stunning images from text prompts! Imagine telling a computer, “Make me a unicorn riding a taco,” and voilà! 🌮🦄
- Turn ONE image into an entire LoRA (LoRA = Low-Rank Adaptation, but just think “superpower upgrade”)
- Create dynamic cinematic videos from images without needing a Hollywood budget 🎬

### Datasets Galore:
- Medical AI datasets that might soon outsmart your doc 🩺
- Reasoning datasets tailored to make bots think harder than your average philosopher 🤔
- Open-source gold mines updated regularly to keep your models sharp and shiny 💎

---

## Why Hugging Face? Glad You Asked!

- **Open & Ethical AI:** We’re championing an AI future that's inclusive and responsible.
- **For Every Modality:** Text, image, video, audio... even 3D objects — we’ve got you covered.
- **Collaboration on Steroids:** Host unlimited public models, datasets, and apps — and share your experiments far and wide.
- **Accelerate Your AI Workflow:** With open-source tools and optional paid compute & enterprise solutions for teams needing more horsepower.

---

## Who’s Hugging Here?

- Aspiring AI scientists, engineers, and artists building their portfolios.
- Hobbyists who want cool AI toys and apps.
- Enterprises ready to supercharge their AI projects with cutting-edge models.
- Innovators who believe in the power of open collaboration and creativity.

---

## Careers: Join the Hug Party! 🎉

Want to build the future of AI in the most welcoming community on the internet? Hugging Face loves curious minds who thrive in open-source culture and enjoy collaboration more than lone wolfin' it. If that sounds like you, get ready to:

- **Collaborate across the globe** with brilliant AI folks.
- **Work on bleeding-edge tools** that move the ML needle.
- **Contribute openly to the community**, helping shape ethical AI.
- Have tons of fun while doing it (because AI is exciting, and so are we).

Keep an eye on their careers page or just show up with your best model and say hello!

---

## Fun Facts to Hug About

- The company colors (#FFD21E & #FF9D00) beam like sunshine, because bright ideas deserve bright colors.
- More than a million models — that’s more AI brainpower than most sci-fi movies combined.
- Hugging Face Spaces = your personal AI playground. No parental controls required.

---

### Ready to Hug the Future?

Whether you’re a data junkie, a curious coder, or an AI enthusiast just looking for cool models to play with — Hugging Face offers you the tools, the community, and the fun environment to shape tomorrow’s technology *today*.

Join the hug. Build something amazing. Change the world with AI — one model at a time.

---

**Hugging Face**: *The AI community building the future — one hug, one model, one dataset at a time.* 🤗

---

[Explore Hugging Face](https://huggingface.co) | [Sign Up for Free] | [Careers & Jobs]

---

*“AI is better when shared. Come build, learn, and play with us!”*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>